In [16]:
## SKLEARN
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

import numpy as np

np.set_printoptions(suppress=True)
import pandas as pd
from platform import python_version

%load_ext nb_black

print("Libs imported. Python version is: " + python_version())

Libs imported. Python version is: 3.9.7


<IPython.core.display.Javascript object>

In [22]:
# utility functions


def load_dataset(x):
    return {"a": load_car(), "b": load_mushroom(), "c": load_audiology(),}[x]


def load_car():
    df_car = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",
        header=None,
    )
    X = df_car.loc[:, :5].values
    y = df_car.loc[:, 6].values


def load_mushroom():
    df_mushroom = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
        header=None,
    )
    X = df_mushroom.loc[:, 1:].values
    y = df_mushroom.loc[:, 0].values
    return (X, y)


def load_audiology():
    df_audiology = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/audiology/audiology.standardized.data",
        header=None,
    )
    length = len(df_audiology.columns)
    X = df_audiology.loc[:, : length - 2].values
    y = df_audiology.loc[:, length - 1].values
    return (X, y)

<IPython.core.display.Javascript object>

In [23]:
# Load dataset
dataset = load_dataset("a")  # CAR
# dataset = load_dataset("b") # MUSHROOM
# dataset = load_dataset("c") # AUDIOLOGY
dataset.

<IPython.core.display.Javascript object>

In [19]:
# print the names of the 13 features
print("Features: ", wine.feature_names)

# print the label type of wine(class_0, class_1, class_2)
print("Labels: ", wine.target_names)

NameError: name 'wine' is not defined

<IPython.core.display.Javascript object>

In [13]:
# print data(feature)shape
wine.data.shape

(178, 13)

In [15]:
# print the wine data features (top 5 records)
print(wine.data[0:2])

[[  14.23    1.71    2.43   15.6   127.      2.8     3.06    0.28    2.29
     5.64    1.04    3.92 1065.  ]
 [  13.2     1.78    2.14   11.2   100.      2.65    2.76    0.26    1.28
     4.38    1.05    3.4  1050.  ]]


In [54]:
# print the wine labels (0:Class_0, 1:class_2, 2:class_2)
print(wine.target)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [6]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.3,random_state=109)

In [7]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [8]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9074074074074074
